# CASO JÓVENES

Queremos resolver dos preguntas relacionadas con los jóvenes (clientes entre 18 y 29 años):											
1ª Pregunta: ¿cómo son?											
2ª Pregunta. ¿cómo han evolucionado respecto a Noviembre 2020?											

### Abrir el fichero de datos

In [20]:
import pandas as pd

In [21]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Transformación Digital/3. CASO JÓVENES.xlsx - BASE CLIENTES NOV21.csv')
df.dtypes

CCLIEN                int64
COD EDAD              int64
COD_TERRIT            int64
SALDO CUENTA          int64
IMPORTE CONSUMO       int64
SEGURO COCHE          int64
IMPORTE HIPOTECA    float64
TENENCIA NOMINA       int64
XTI_IDEFISCO          int64
dtype: object

#### Filtros 
Es un problema de clientes particulares, así tendremos que filtrar a las personas jurídicas, que si comprobamos en el catálogo, se seleccionan por XTI_IDEFISCO = 2 Ó 3											

In [22]:
print('Número de registros: ', df.shape[0])
df_filtro_partic = df[(df['XTI_IDEFISCO'] != 2) & (df['XTI_IDEFISCO'] != 3)]
print('Número de registros filtrando por particulares: ', df_filtro_partic.shape[0])

Número de registros:  1000
Número de registros filtrando por particulares:  915


#### Limpieza de datos
Comprobamos si hay missing values, sustituyéndolos por 0

In [23]:
df_filtro_partic.isna().sum()

CCLIEN              0
COD EDAD            0
COD_TERRIT          0
SALDO CUENTA        0
IMPORTE CONSUMO     0
SEGURO COCHE        0
IMPORTE HIPOTECA    4
TENENCIA NOMINA     0
XTI_IDEFISCO        0
dtype: int64

In [24]:
df_limpio = df_filtro_partic.fillna(0)

In [25]:
df_limpio.isna().sum()

CCLIEN              0
COD EDAD            0
COD_TERRIT          0
SALDO CUENTA        0
IMPORTE CONSUMO     0
SEGURO COCHE        0
IMPORTE HIPOTECA    0
TENENCIA NOMINA     0
XTI_IDEFISCO        0
dtype: int64

#### Variables contador
* Generamos variables contador de tenencia de productos y de hipoteca
* Generamos variable de tramo de edad: menores, jóvenes, entre 30 y 59 y mayores de 59 años

In [26]:
import numpy as np

df_limpio['HIPOTECA'] = np.where(df_limpio['IMPORTE HIPOTECA'] > 0, 1, 0)
df_limpio['CONSUMO'] = np.where(df_limpio['IMPORTE CONSUMO'] > 0, 1, 0)

In [27]:
conditions = [
    (df_limpio['COD EDAD'] < 18),
    (df_limpio['COD EDAD'] >= 18) & (df_limpio['COD EDAD'] < 30),
    (df_limpio['COD EDAD'] >= 30) & (df_limpio['COD EDAD'] <= 59),
    (df_limpio['COD EDAD'] > 59),
]

choices = ['A', 'B', 'D', 'C']

df_limpio['TRAMO_EDAD'] = np.select(conditions, choices, default=0)
df_limpio['TRAMO_EDAD']

0      B
1      C
2      B
3      B
4      B
      ..
995    B
996    D
997    B
998    D
999    B
Name: TRAMO_EDAD, Length: 915, dtype: object

In [28]:
df_limpio.groupby('TRAMO_EDAD').agg({'TRAMO_EDAD': 'count', 
                                     'TENENCIA NOMINA': 'sum',
                                     'IMPORTE CONSUMO':'sum',
                                     'CONSUMO': 'sum',
                                     'IMPORTE HIPOTECA':'sum',
                                     'HIPOTECA': 'sum',
                                     'SEGURO COCHE': 'sum',
                                     'SALDO CUENTA': 'sum'})

,TRAMO_EDAD,TENENCIA NOMINA,IMPORTE CONSUMO,CONSUMO,IMPORTE HIPOTECA,HIPOTECA,SEGURO COCHE,SALDO CUENTA
TRAMO_EDAD,,,,,,,,
A,57,0,0,0,0.0,0,0,96100
B,426,136,1283594,237,9299289.0,55,118,2014668
C,157,21,252730,50,3739769.0,19,36,8259325
D,275,142,847728,165,24092870.0,140,96,13186452


In [29]:
df_limpio.groupby('TRAMO_EDAD').agg({'TRAMO_EDAD': lambda x: x.count() / df_limpio.shape[0] * 100,
                                     'TENENCIA NOMINA': lambda x: x.sum() / float(x.count()) * 100,
                                     'CONSUMO': lambda x: x.sum() / float(x.count()) * 100,
                                     'HIPOTECA': lambda x: x.sum() / float(x.count()) * 100,
                                     'SEGURO COCHE': lambda x: x.sum() / float(x.count()) * 100,
                                     'SALDO CUENTA': lambda x: x.sum() / float(x.count())})

,TRAMO_EDAD,TENENCIA NOMINA,CONSUMO,HIPOTECA,SEGURO COCHE,SALDO CUENTA
TRAMO_EDAD,,,,,,
A,6.229508,0.000000,0.000000,0.000000,0.000000,1685.964912
B,46.557377,31.924883,55.633803,12.910798,27.699531,4729.267606
C,17.158470,13.375796,31.847134,12.101911,22.929936,52607.165605
D,30.054645,51.636364,60.000000,50.909091,34.909091,47950.734545


* 1ª Conclusión: los jóvenes suponen un 46,56% de mi base de clientes
* 2ª Conclusión: destaca su tenencia de consumo, así como su tenencia de seguro de coche

#### Cruce de las bases de datos

Vamos a calcular cómo ha evolucionado mi base de jóvenes respecto a Noviembre 2020 (sin filtros)

In [30]:
df_2021 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Transformación Digital/3. CASO JÓVENES.xlsx - BASE CLIENTES NOV21.csv')
print(df_2021.shape)

df_2020 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Transformación Digital/3. CASO JÓVENES.xlsx - BASE CLIENTES NOV20.csv')
print(df_2020.shape)

(1000, 9)
(949, 9)


In [31]:
result = pd.merge(
    df_2021,
    df_2020,
    how="inner",
    on="CCLIEN")
print("Numero de clientes que cruzan: ", result.shape[0])

Numero de clientes que cruzan:  889


1) En primer lugar vamos a ver cómo ha evolucionado la base completa de clientes y posteriormente veremos cómo ha evolucionado la base de jóvenes

In [32]:
print('ALTAS: ', df_2021.shape[0] - result.shape[0])
print('BAJAS: ', df_2020.shape[0] - result.shape[0])
print('PERMANECEN: ', result.shape[0])

ALTAS:  111
BAJAS:  60
PERMANECEN:  889


In [33]:
result.columns

Index(['CCLIEN', 'COD EDAD_x', 'COD_TERRIT_x', 'SALDO CUENTA_x',
       'IMPORTE CONSUMO_x', 'SEGURO COCHE_x', 'IMPORTE HIPOTECA_x',
       'TENENCIA NOMINA_x', 'XTI_IDEFISCO_x', 'COD EDAD_y', 'COD_TERRIT_y',
       'SALDO CUENTA_y', 'IMPORTE CONSUMO_y', 'SEGURO COCHE_y',
       'IMPORTE HIPOTECA_y', 'TENENCIA NOMINA_y', 'XTI_IDEFISCO_y'],
      dtype='object')

In [34]:
df_2021['JOVEN'] = np.where((df_2021['COD EDAD'] >= 18) & (df_2021['COD EDAD'] < 30) & (df_2021['XTI_IDEFISCO'] != 2) & (df_2021['XTI_IDEFISCO'] != 3), 1, 0)
df_2020['JOVEN'] = np.where((df_2020['COD EDAD'] >= 18) & (df_2020['COD EDAD'] < 30) & (df_2020['XTI_IDEFISCO'] != 2) & (df_2020['XTI_IDEFISCO'] != 3), 1, 0)
result['JOVEN'] = np.where((result['COD EDAD_x'] >= 18) & (result['COD EDAD_x'] < 30) & (result['XTI_IDEFISCO_x'] != 2) & (result['XTI_IDEFISCO_x'] != 3), 1, 0)

df_2021_jovenes = df_2021[df_2021['JOVEN'] == 1]
df_2020_jovenes = df_2020[df_2020['JOVEN'] == 1]
result_jovenes = result[result['JOVEN'] == 1]

print('JOVENES - 2021: ', df_2021_jovenes.shape[0])
print('JOVENES - 2020: ', df_2020_jovenes.shape[0])
print('JOVENES QUE PERMANECEN: ', result_jovenes.shape[0])
print('ALTAS: ', df_2021_jovenes.shape[0] - result_jovenes.shape[0])
print('BAJAS: ', df_2020_jovenes.shape[0] - result_jovenes.shape[0])

JOVENES - 2021:  426
JOVENES - 2020:  396
JOVENES QUE PERMANECEN:  389
ALTAS:  37
BAJAS:  7


Se cumple el cálculo: Base Nov20 - Bajas = Clientes que permanecen o Base Nov21 - Altas = Clientes que permanecen

2) Vamos a hacer el mismo cálculo con los Jóvenes

In [35]:
df_all = df_2020.merge(df_2021, on=['CCLIEN'], 
                   how='outer', indicator=True)
print("Numero de clientes que cruzan: ", df_all.shape[0])
df_all['JOVEN_x'] = np.where((df_all['COD EDAD_x'] >= 18) & (df_all['COD EDAD_x'] < 30) & (df_all['XTI_IDEFISCO_x'] != 2) & (df_all['XTI_IDEFISCO_x'] != 3), 1, 0)
df_x = df_all[df_all['JOVEN_x'] == 1]
df_x.groupby('_merge').agg({'_merge': 'count'})

Numero de clientes que cruzan:  1060


,_merge
_merge,
left_only,30
right_only,0
both,366


In [36]:
df_all['JOVEN_y'] = np.where((df_all['COD EDAD_y'] >= 18) & (df_all['COD EDAD_y'] < 30) & (df_all['XTI_IDEFISCO_y'] != 2) & (df_all['XTI_IDEFISCO_y'] != 3), 1, 0)
df_y = df_all[df_all['JOVEN_y'] == 1]
df_y.groupby('_merge').agg({'_merge': 'count'})

,_merge
_merge,
left_only,0
right_only,37
both,389


¿Por qué hay dos valores posibles para clientes que permanecen?¿por qué no se cumplen los cálculos anteriores?
Porque nos falta considerar las altas vegetativas / bajas vegetativas: los que cumplen 18 o cumplen 30

In [40]:
df_bajas_vegetativas = df_all[(df_all['COD EDAD_x'] < 30) & (df_all['COD EDAD_y'] >= 30) & (df_all['XTI_IDEFISCO_y'] != 2) & (df_all['XTI_IDEFISCO_y'] != 3)]
print('BAJAS VEGETATIVAS: ', df_bajas_vegetativas.shape[0])

BAJAS VEGETATIVAS:  8


In [39]:
df_altas_vegetativas = df_all[(df_all['COD EDAD_x'] < 18) & (df_all['COD EDAD_y'] >= 18) & (df_all['XTI_IDEFISCO_y'] != 2) & (df_all['XTI_IDEFISCO_y'] != 3)]
print('ALTAS VEGETATIVAS: ', df_altas_vegetativas.shape[0])

ALTAS VEGETATIVAS:  31


Resolución: el buen crecimiento de la base de jóvenes se debe fundamentalmente a las altas vegetativas ya que hay un volumen de altas similar al de bajas, pero más altas vegetativas que bajas vegetativas. Esto tiene un riesgo: si no se cuida la base de menores, este crecimiento no se seguirá produciendo									